In [ ]:
import tensorflow as tf

# Load the SavedModel
loaded_model = tf.saved_model.load('./saved_model/')

# Define a sample input (replace with your actual input shape)
sample_input = tf.ones((1, 200, 200, 1), dtype=tf.float32)

# Create a concrete function using tf.function
@tf.function(input_signature=[tf.TensorSpec(shape=(None, 200, 200, 1), dtype=tf.float32)])
def inference_fn(inputs):
    return loaded_model(inputs)

# Get the concrete function
concrete_func = inference_fn.get_concrete_function(sample_input)

# Convert the model to a frozen graph
frozen_graph_def = tf.compat.v1.graph_util.convert_variables_to_constants(
    tf.compat.v1.Session(graph=tf.Graph()), 
    concrete_func.graph.as_graph_def(), 
    [concrete_func.outputs[0].name.split(":")[0]]
)

# Save the frozen graph to a file
with tf.io.gfile.GFile('frozen_model.pb', 'wb') as f:
    f.write(frozen_graph_def.SerializeToString())
